In [20]:
import pandas as pd
import numpy as np


def generate_arrays_from_file(path, batch_size, each_scholar_vectorLen):
    while 1:
        f = open(path)
        data_batch = []
        label_batch = []
        for line in f:
            tmp = line.split()

            label_batch.append(int(tmp[0]))
            if len(tmp) > each_scholar_vectorLen + 2:
                data = data[0:each_scholar_vectorLen]
            else:
                data.extend([0]*(each_scholar_vectorLen - len(data) + 2))
            data_batch.append(data[2:])

            
            
            yield(y)
    f.close
g = generate_arrays_from_file("dataRecord_vector_2_397.txt", 1)
print(next(g))
print(next(g))
print(next(g))


['1' 'TULvhq0AAAAJ' '20220515' '4245' '1']
['1' 'pYZos38AAAAJ' '20220515' '3998' '1']
['1' 'cakwOcQAAAAJ' '20220515' '3676' '1']


StopIteration: 

In [16]:
from keras.layers import Dense, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
import numpy as np
# parameters for LSTM
nb_lstm_outputs = 1    # 輸出神經元個數
nb_time_steps = 1    # 時間序列的長度
nb_input_vectors =  100 # 每個輸入序列的向量維度

def generate_arrays_from_file(path, batch_size, each_scholar_vectorLen):
    while 1:
        f = open(path)
        cnt = 0
        data_batch = []
        label_batch = []
        for index, line in enumerate(f):
            data = line.split()

            label_batch.append(int(data[0]))
            # 判斷向量的長度，多的去掉，少的補0
            if len(data) > each_scholar_vectorLen + 2:
                data = data[0:each_scholar_vectorLen + 2]
            else:
                data.extend([0]*(each_scholar_vectorLen - len(data) + 2))
            data_batch.append(data[2:]) # data[0] : label, data[1] : ID
            cnt += 1
            if (cnt == batch_size):
                cnt = 0
                dataset = np.array(data_batch)
                dataset = dataset.astype('float32')
                # 正規化，使資料值介於[0, 1]
                #scaler = MinMaxScaler(feature_range=(0, 1))
                #dataset = scaler.fit_transform(dataset)
                train = np.reshape(dataset, (dataset.shape[0], 1, dataset.shape[1]))
                label = np.array(label_batch)
                print(f"\nindex: {index} trainX: {train.shape},labelX: {label.shape}")
                yield(train, label)
                data_batch = []
                label_batch = []
        f.close()        

# building model
model = Sequential()

model.add(
    Bidirectional(
        LSTM(units=64, return_sequences=True),#LSTM(units=nb_lstm_outputs, return_sequences=True),
        input_shape=(nb_time_steps, nb_input_vectors,)
    )
)
model.add(Bidirectional(LSTM(units=32)))

model.add(Dense(16, activation="relu"))

model.add(Dense(nb_lstm_outputs, activation='relu'))

# compile:loss, optimizer, metrics
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

model.summary()

model.fit_generator(generate_arrays_from_file("dataRecord_vector_2_397.txt", 1, 100), steps_per_epoch=4, epochs=2, verbose=2)


trainX: (1, 1, 100),labelX: (1,)
trainX: (2, 1, 100),labelX: (2,)
trainX: (3, 1, 100),labelX: (3,)


In [12]:
def generate_dataframe(filename, each_scholar_vectorLen):
    with open(filename, "r") as file:
        dataList = []
        labelList = []
        for index, line in enumerate(file):
            if index > 9000:
                break
            data = line.split()
            labelList.append(int(data[0]))
            if len(data) > each_scholar_vectorLen + 2:
                data = data[2:each_scholar_vectorLen]
            else:
                data.extend([0]*(each_scholar_vectorLen - len(data) + 2) )
                data = data[2:]
            dataList.append(data)
    
    df = pd.DataFrame(dataList)
    label_df = pd.DataFrame(labelList)
    return df, label_df

dataframe, label_df = generate_dataframe("./dataRecord_vector_2_397.txt", 100)

dataset = dataframe.values
dataset = dataset.astype('float32')
print(dataset)

[[ 2.02205160e+07  4.24500000e+03  1.00000000e+00  2.98978877e+00
   6.42452955e-01 -7.12157130e-01  1.79953182e+00  6.42616212e-01
   4.88736439e+00  3.71095514e+00 -3.72403592e-01  2.31035018e+00
  -7.44502783e-01 -7.18392134e-01  1.02630806e+00  2.32723475e+00
  -2.95525074e-01  4.63386345e+00  4.48978567e+00  5.02468109e+00
   1.24918640e+00 -4.00306404e-01  2.58551002e+00  1.05525458e+00
   2.46885598e-01  7.84803107e-02  2.64583254e+00  5.34706306e+00
   2.60637939e-01  1.50994849e+00  1.50543916e+00  3.55844522e+00
   7.48270893e+00  8.13985109e-01  7.25119472e-01  9.78735864e-01
   5.70415735e+00 -6.90134346e-01  1.08077653e-01  4.33313036e+00
  -6.83723211e-01  1.10592818e+00  5.70816159e-01  1.27882791e+00
   7.76474833e-01  2.94348001e+00  1.09244001e+00  2.08178926e+00
   1.32866874e-02  1.13289319e-01  6.75930071e+00  1.70083046e+01
   6.23792362e+00  3.70951557e+00  1.34713435e+00 -5.87040126e-01
   2.74204230e+00  2.76170349e+00  1.27002325e+01  4.40208226e-01
   6.50195